In [ ]:
import pandas as pd
from preprocessing import remove_incomplete_days
import pandas as pd
from sklearn.preprocessing import minmax_scale
import torch

loadProfiles_df = pd.read_parquet(r'data/load_profiles.parquet.gzip')
loadProfiles_df = remove_incomplete_days(loadProfiles_df)

df = loadProfiles_df.pivot_table(values = '1163', index = ['date', 'month of the year', 'day off'], columns = 'hour of the day').reset_index()
labels = list((df['month of the year'].astype(str) + df['day off'].astype(str)).astype(int))
df.drop(['date', 'month of the year', 'day off'], axis = 1, inplace = True)
df = pd.DataFrame(minmax_scale(df, feature_range = (-0.5, 0.5), axis = 1))
timeSeries = df.values.tolist()
trainloader = torch.utils.data.DataLoader([[timeSeries[i], labels[i]] for i in range(len(labels))], shuffle = True, batch_size = 100)

In [ ]:
import torch.nn as nn
import torch
import numpy as np

# %% Generator
class Generator(nn.Module):
    def __init__(self, classCount, dimLatent):
        super(Generator, self).__init__()
        self.classCount = classCount
        self.dimLatent = dimLatent
        self.labelEmbedding = nn.Embedding(num_embeddings = self.classCount, embedding_dim = self.classCount)
        self.img_shape = (1, 24)
        self.model = nn.Sequential(
            # 1st layer
            nn.Linear(in_features = self.classCount + self.dimLatent, out_features = 64),
            nn.LeakyReLU(),
            # 2nd layer
            nn.Linear(in_features = 64, out_features = 128),
            nn.LeakyReLU(),
            # 3rd layer
            nn.BatchNorm1d(num_features = 128, momentum = 0.8),
            # 4th layer
            nn.Linear(in_features = 128, out_features = 64),
            nn.LeakyReLU(),
            # 5th layer
            nn.Linear(in_features = 64, out_features = 24)
        )
    
    def forward(self, noise, labels):
        z = torch.cat((self.labelEmbedding(labels), noise), -1)
        x = self.model(z)
        x = x.view(x.size(0), *self.img_shape)
        return x

# %% Discriminator
class Discriminator(nn.Module):
    def __init__(self, classCount, featureCount):
        super(Discriminator, self).__init__()
        self.classCount = classCount
        self.featureCount = featureCount
        self.labelEmbedding = nn.Embedding(num_embeddings = self.classCount, embedding_dim = self.classCount)
        self.adv_loss = torch.nn.BCELoss()
        self.model = nn.Sequential(
            # 1st layer
            nn.Linear(in_features = self.classCount + self.featureCount, out_features = 64),
            nn.LeakyReLU(),
            # 2nd layer
            nn.Linear(in_features = 64, out_features = 128),
            nn.LeakyReLU(),
            # 3rd layer
            nn.Linear(in_features = 128, out_features = 128),
            nn.LeakyReLU(),
            # 4th layer
            nn.Linear(in_features = 128, out_features = 64),
            nn.LeakyReLU(),
            # 5th layer
            nn.Dropout(p = 0.4),
            # 6th layer
            nn.Linear(in_features = 64, out_features = 1),
            nn.Sigmoid()
        )
    
    def forward(self, timeSeries, labels):
        x = torch.cat((timeSeries.view(timeSeries.size(0), -1), self.labelEmbedding(labels)), -1)
        x = x.to(torch.float32)
        return self.model(x)
    
    def loss(self, output, label):
        return self.adv_loss(output, label)

# %% GAN
class GAN(object):
    def __init__(self, device, dataLoader, classCount, dimLatent, featureCount):
        self.device = device
        self.dataLoader = dataLoader
        self.classCount = classCount
        self.dimLatent = dimLatent
        self.featureCount = featureCount
        self.netGen = Generator(classCount, dimLatent)
        self.netGen.to(self.device)
        self.netDis = Discriminator(classCount, featureCount)
        self.netDis.to(self.device)
    
    def create_optim(self, lr):
        self.optimGen = torch.optim.Adam(params = self.netGen.parameters(), lr = lr)
        self.optimDis = torch.optim.Adam(params = self.netDis.parameters(), lr = lr)
    
    def to_onehot(self, var, dim):
        res = torch.zeros((var.shape[0], dim), device = self.device)
        res[range(var.shape[0]), var] = 1.
        return res

    def train(self, epochs):
        self.netGen.train()
        self.netDis.train()
        for item in range(epochs):
            for batchIdx, (data, target) in enumerate(self.dataLoader):
                data = torch.stack(data)
                data = torch.tensor(data)
                target = torch.tensor(target)
                data, target = data.T.to(self.device), target.to(self.device)
                batchSize = data.size(0)
                print(batchSize)
                labelReal = torch.full((batchSize, 1), 1., device = self.device)
                labelFake = torch.full((batchSize, 1), 0., device = self.device)
                # Train generator
                self.netGen.zero_grad()
                zNoise = torch.randn(batchSize, self.dimLatent, device = self.device)
                xLabelsFake = torch.randint(0, self.classCount, (batchSize,), device = self.device)
                xFake = self.netGen(zNoise, xLabelsFake)
                yFakeGen = self.netDis(xFake, xLabelsFake)
                lossGen = self.netDis.loss(yFakeGen, labelReal)
                self.optimGen.step()
                # Train discriminator
                self.netDis.zero_grad()
                yReal = self.netDis(data, target)
                realLossDis = self.netDis.loss(yReal, labelReal)
                yFakeDis = self.netDis(xFake.detach(), xLabelsFake)
                fakeLossDis = self.netDis.loss(yFakeDis, labelFake)
                lossDis = (realLossDis + fakeLossDis)/2
                lossDis.backward()
                self.optimDis.step()

In [ ]:
def make():
    device = torch.device('cuda:0')
    model = GAN(device, trainloader, 24, 20, 24)
    model.create_optim(lr = 4e-4)
    model.train(epochs = 10)

make()